In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import csv


In [2]:

# ---- SETUP ---- #
CHROMEDRIVER_PATH = "/Users/parakhchaudhary/chromedriver_135/chromedriver-mac-x64/chromedriver"

options = Options()
options.add_argument("--headless")  # Optional: remove to view browser
service = Service(CHROMEDRIVER_PATH)
driver = webdriver.Chrome(service=service, options=options)

# Visit the product catalog main page
driver.get("https://www.shl.com/products/product-catalog/?type=1")
time.sleep(3)

all_data = []


In [ ]:

for _ in range(13):  # Go to page 14
    next_buttons = driver.find_elements(By.XPATH, '//a[contains(text(), "Next")]')
    if len(next_buttons) > 1:
        driver.execute_script("arguments[0].click();", next_buttons[1])  # second = individual
    else:
        driver.execute_script("arguments[0].click();", next_buttons[0])
    time.sleep(2)  # give content time to load


In [4]:

seen_first_test_name = None

page_num = 14
while True:
    print(f"Scraping page {page_num}")
    soup = BeautifulSoup(driver.page_source, "html.parser")

    tables = soup.find_all("table")
    if not tables:
        print("❌ No table found.")
        break

    table = tables[-1]  # Always get the last (individual test) table
    rows = table.find_all("tr")[1:]
    if not rows:
        break

    # Check first row's test name to detect repetition
    a_tag = rows[0].find_all("td")[0].find("a")
    current_first_name = a_tag.text.strip()

    if current_first_name == seen_first_test_name:
        print("⚠️ Same content as previous page. Stopping.")
        break

    seen_first_test_name = current_first_name

    # Parse all rows on current page
    for row in rows:
        cols = row.find_all("td")
        if len(cols) < 4:
            continue
        a_tag = cols[0].find("a")
        name = a_tag.text.strip()
        href = a_tag['href'].strip()
        remote = "Yes"
        adaptive = "Yes" if cols[2].find("svg") else "No"
        test_type = ''.join(span.text for span in cols[3].find_all("span"))

        all_data.append({
            "name": name,
            "url": href,
            "remote_testing": remote,
            "adaptive_support": adaptive,
            "test_type": test_type
        })

    # Try clicking "Next"
    try:
        next_buttons = driver.find_elements(By.XPATH, '//a[contains(text(), "Next")]')

        # Always pick the second one (index 1) on page 1
        if len(next_buttons) > 1 and page_num == 1:
            next_button = next_buttons[1]
        else:
            next_button = next_buttons[0]
        driver.execute_script("arguments[0].click();", next_button)

        # Wait until first test name changes
        WebDriverWait(driver, 10).until(
            lambda d: current_first_name not in d.page_source
        )
        page_num += 1
        time.sleep(1)
    except:
        print("✅ Reached last page.")
        break


Scraping page 14
Scraping page 15
Scraping page 16
Scraping page 17
Scraping page 18
Scraping page 19
Scraping page 20
Scraping page 21
Scraping page 22
Scraping page 23
Scraping page 24
Scraping page 25
Scraping page 26
Scraping page 27
Scraping page 28
Scraping page 29
Scraping page 30
Scraping page 31
Scraping page 32
Scraping page 33
Scraping page 34
Scraping page 35
Scraping page 36
Scraping page 37
Scraping page 38
Scraping page 39
❌ No table found.


In [5]:

# Close the browser
driver.quit()


In [6]:

# ---- SAVE TO CSV ---- #
output_file = "shl_catalog3_31onwards.csv"
with open(output_file, "w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=all_data[0].keys())
    writer.writeheader()
    writer.writerows(all_data)

print(f"✅ Done. Scraped {len(all_data)} assessments and saved to {output_file}")


✅ Done. Scraped 300 assessments and saved to shl_catalog3_31onwards.csv
